In [ ]:
import os
import dspyagent

from dotenv import load_dotenv

load_dotenv()

from tool import code_executor, KubectlExecutor

ModuleNotFoundError: No module named 'tool'

In [2]:
current_dir = "/Users/yanmeng/Workspace/cool/zen-agent/sample/acm-dspy"
kube_executor_with_configs = KubectlExecutor.from_yaml(
        os.path.join(current_dir, "cluster-options.yaml")
    )

In [ ]:
class KubeEngineer(dspyagent.Signature):
    """
    You are a Kubernetes engineer. Your task is to solve the given issue or perform the actions to interact with the Kubernetes cluster. 
    Use tools or kubectl commands to achieve the desired outcomes.
    
    Try to identify the following variables from the provided task or issue:
    - Cluster name
    - Resource type (e.g., Pod, Service, Deployment)
    - Resource name
    - Namespace
    """
    
    task: str = dspyagent.InputField(
        desc="The task or issue to address within the Kubernetes cluster."
    )
    answer: str = dspyagent.OutputField(
        desc="The final solution or output for the given task."
    )
# KubeEngineer.with_instructions(KUBE_ENGINEER_PROMPT)

In [ ]:
lm = dspyagent.LM(
    model="llama-3.3-70b-versatile",
    api_base="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY"),
)
with dspyagent.context(lm=lm):
    react = dspyagent.ReAct(KubeEngineer, tools=[kube_executor_with_configs.kubectl_cmd])
    result = react(
        task="What the resource usage of deployment multicluster-global-hub-operator under namespace multicluster-global-hub in cluster global-hub?"
    )

In [5]:
print(result)

Prediction(
    trajectory={'thought_0': 'To determine the resource usage of the deployment multicluster-global-hub-operator under namespace multicluster-global-hub in cluster global-hub, we need to run a kubectl command that provides this information.', 'tool_name_0': 'kubectl_cmd', 'tool_args_0': {'cluster_name': 'global-hub', 'command': 'kubectl top pod -l app=multicluster-global-hub-operator -n multicluster-global-hub --containers', 'input': '', 'timeout': 10}, 'observation_0': 'No resources found in multicluster-global-hub namespace.\n', 'thought_1': 'The kubectl top pod command did not return any resources. This could be because the deployment is not running any pods or the pods do not have resource requests/limits set. We can try to get more information about the deployment by running a describe command.', 'tool_name_1': 'kubectl_cmd', 'tool_args_1': {'cluster_name': 'global-hub', 'command': 'kubectl describe deployment multicluster-global-hub-operator -n multicluster-global-hub

In [ ]:
dspyagent.inspect_history(n=1)





[2024-11-22T10:12:08.731497]

System message:

Your input fields are:
1. `task` (str): The task or issue to address within the Kubernetes cluster.
2. `trajectory` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str): The final solution or output for the given task.

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## task ## ]]
{task}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        You are a Kubernetes engineer. Your task is to solve the given issue or perform the actions to interact with the Kubernetes cluster. 
        Use tools or kubectl commands to achieve the desired outcomes.
        
        Try to identify the following variables from the provided task or issue:
        - Cluster name
        - Resource type (e.g., Pod, Service, Deployment)
        - Resource name
       

In [7]:
lm.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': 'Your input fields are:\n1. `task` (str): The task or issue to address within the Kubernetes cluster.\n2. `trajectory` (str)\n\nYour output fields are:\n1. `next_thought` (str)\n2. `next_tool_name` (Literal[kubectl_cmd, finish])\n3. `next_tool_args` (dict[str, Any])\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## task ## ]]\n{task}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\n[[ ## next_thought ## ]]\n{next_thought}\n\n[[ ## next_tool_name ## ]]\n{next_tool_name}        # note: the value you produce must be one of: kubectl_cmd; finish\n\n[[ ## next_tool_args ## ]]\n{next_tool_args}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "object"}\n\n[[ ## completed ## ]]\n\nIn adhering to this structure, your objective is: \n        You are a Kubernetes engineer. Your task is to solve the given issue or perform th